### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
!pip install bs4
import bs4 as bs
import urllib.request
import pandas as pd 
l = []
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html5lib')
table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    l.append(row)
r = pd.DataFrame(l, columns=["Postalcode", "Borough","Neighbourhood"])
r.reset_index(inplace = True) 

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
##Removing the not assigned values in Borough

r = r[r['Borough'] != 'Not assigned']
print(r)

     index Postalcode           Borough               Neighbourhood
0        0       None              None                        None
3        3        M3A        North York                 Parkwoods\n
4        4        M4A        North York          Victoria Village\n
5        5        M5A  Downtown Toronto              Harbourfront\n
6        6        M6A        North York          Lawrence Heights\n
..     ...        ...               ...                         ...
282    282        M8Z         Etobicoke  Kingsway Park South West\n
283    283        M8Z         Etobicoke                 Mimico NW\n
284    284        M8Z         Etobicoke        The Queensway West\n
285    285        M8Z         Etobicoke     Royal York South West\n
286    286        M8Z         Etobicoke            South of Bloor\n

[211 rows x 4 columns]


### Two rows will be combined into one row with the neighborhoods separated with a comma when single postal code has has two neighborhoods

In [4]:
r1=r.groupby("Postalcode").agg(lambda x:','.join(set(x)))

r1.reset_index(inplace = True) 
r1
###set A Set is an unordered collection data type that is iterable, mutable, and has no duplicate elements. 

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge\n,Malvern\n"
1,M1C,Scarborough,"Port Union\n,Highland Creek\n,Rouge Hill\n"
2,M1E,Scarborough,"West Hill\n,Morningside\n,Guildwood\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
...,...,...,...
98,M9N,York,Weston\n
99,M9P,Etobicoke,Westmount\n
100,M9R,Etobicoke,"Richview Gardens\n,Kingsview Village\n,Martin ..."
101,M9V,Etobicoke,"Humbergate\n,Mount Olive\n,South Steeles\n,Sil..."


### if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [5]:
r1.loc[r1['Neighbourhood']=="Not assigned",'Neighbourhood']=r1.loc[r1['Neighbourhood']=="Not assigned",'Borough']

### using the shape method 

In [6]:
r1.shape

(103, 3)

In [8]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
r1['Latitude'] =geo_data['Latitude'].values
r1['Longitude'] =geo_data['Longitude'].values

In [47]:
r1

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Port Union\n,Highland Creek\n,Rouge Hill\n",43.784535,-79.160497
2,M1E,Scarborough,"West Hill\n,Morningside\n,Guildwood\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston\n,43.706876,-79.518188
99,M9P,Etobicoke,Westmount\n,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens\n,Kingsview Village\n,Martin ...",43.688905,-79.554724
101,M9V,Etobicoke,"Humbergate\n,Mount Olive\n,South Steeles\n,Sil...",43.739416,-79.588437


In [21]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
center_lat=r1.Latitude.mean()
center_long=r1.Longitude.mean()

venues_map = folium.Map(location=[center_lat, center_long], zoom_start=13) # generate map centred around the Conrad Hotel
df1 = r1[r1.Borough.str.contains('Toronto')]

for lat,lng,label in zip(df1.Latitude,df1.Longitude,df1.Borough):
    folium.features.CircleMarker(
               [lat,lng],
            radius=2,
            color ='blue',
            fill =True,
            ill_color='blue',
            fill_opacity=0.6
    ).add_to(venues_map)
    
venues_map

